In [26]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [27]:
# poetry add pydantic
# %pip install pydantic 

from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema import OutputParserException

from pydantic import BaseModel, Field
from typing import List

In [42]:
# 출력 구조를 정의하는 Pydantic 모델
class StudentInfos(BaseModel):
    name: str = Field(description="이름")
    age: int = Field(description="나이")
    major: str  = Field(description="전공")
    hobbies: List[str] = Field(description="취미")
    goal: str = Field(description="목표")
    
# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=StudentInfos)
format_instructions = parser.get_format_instructions()

# 프롬프트 템플릿 설정
template = """
다음 학생의 자기소개에서 이름, 나이, 전공, 취미, 목표를 추출하고 JSON 형식으로 출력하세요.

요청: {query}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(format_instructions=format_instructions)



In [39]:
# ChatOpenAI 모델 초기화
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 구성 및 실행
query =  "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."
chain = prompt | model | parser
output = chain.invoke({"query": query})
# 결과 출력
try:
    print(f"이름: {output.name}")
    print(f"나이: {output.age}")
    print(f"전공: {output.major}")
    print(f"취미: {', '.join(output.hobbies)}")
    print(f"목표: {output.goal}")
except OutputParserException as e:
    print("출력 파싱 오류:", e)
    print("원본 출력:", e.output)  # 어떤 출력이 문제였는지 확인

이름: 김민수
나이: 22
전공: 컴퓨터공학
취미: 게임하기, 영화보기, 코딩
목표: 훌륭한 개발자가 되는 것
